In [2]:
import pandas as pd
import numpy as np
import requests
import plotly.express as px
import plotly.graph_objects as go

# Configuration
BIDDING_ZONE = "DK1"
API_URL = "https://api.energidataservice.dk/dataset/PowerSystemRightNow"

# Set default template for a clean look
import plotly.io as pio
pio.templates.default = "plotly_white"

In [8]:
def fetch_history(zone, days_back=7):
    """
    Downloads historical aFRR activation data.
    """
    limit = days_back * 24 * 60
    
    params = {
        "limit": limit,
        "sort": "Minutes1UTC DESC" 
    }
    
    print(f"Fetching {days_back} days of data for {zone}...")
    response = requests.get(API_URL, params=params)
    response.raise_for_status()
    data = response.json()
    
    df = pd.DataFrame(data['records'])
    
    # Process Timestamp (UTC -> CET)
    df.index = pd.to_datetime(df['Minutes1UTC']).dt.tz_localize("UTC").dt.tz_convert("CET")
    df.sort_index(inplace=True)
    
    # Select specific column
    col_name = f"aFRR_Activated{zone}"
    series = df[col_name].astype(float)
    series.name = "Activation_MW"
    
    print(f"Loaded {len(series)} data points.")
    return series

# Fetch Data
df_activations = fetch_history(BIDDING_ZONE, days_back=7)
df_clean = df_activations.interpolate(method='time')
df_clean.head()

Fetching 7 days of data for DK1...
Loaded 10080 data points.


Minutes1UTC
2026-01-21 21:55:00+01:00   -209.970001
2026-01-21 21:56:00+01:00   -212.720001
2026-01-21 21:57:00+01:00   -207.279999
2026-01-21 21:58:00+01:00   -200.149994
2026-01-21 21:59:00+01:00   -206.889999
Name: Activation_MW, dtype: float64

In [10]:
# 1. Full History Plot
fig = px.line(df_clean, y="Activation_MW", title=f"aFRR Activation {BIDDING_ZONE} - Full History")
fig.update_traces(line_color='royalblue', line_width=1)
fig.add_hline(y=0, line_dash="dash", line_color="black")


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Minutes1UTC=%{x}<br>Activation_MW=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': 'royalblue', 'dash': 'solid', 'width': 1},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '',
              'showlegend': False,
              'type': 'scattergl',
              'x': array(['2026-01-21T21:55:00.000000000', '2026-01-21T21:56:00.000000000',
                          '2026-01-21T21:57:00.000000000', ..., '2026-01-28T21:53:00.000000000',
                          '2026-01-28T21:54:00.000000000', '2026-01-28T21:55:00.000000000'],
                         shape=(10080,), dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('y4KJPwo/asDLgok/CpdqwDV9dsD16G' ... 'XB/9bjZMDfFcH/1uNkwKuYSj/h+mbA'),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'shapes': [{'line': {'color': 'black', 'dash': 'dash'},
                           'type': 'line',
                           'x0': 0,
                           'x1': 1,
                           'xref': 'x domain',
                           'y0': 0,
                           'y1': 0,
                           'yref': 'y'}],
               'template': '...',
               'title': {'text': 'aFRR Activation DK1 - Full History'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Minutes1UTC'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Activation_MW'}}}
})

In [11]:


# 2. Zoom on Last 48 Hours
last_48h = df_clean.iloc[-2880:]
fig_zoom = px.line(last_48h, y="Activation_MW", title="Zoom: Last 48 Hours Microstructure")
fig_zoom.update_traces(line_color='darkorange', line_width=1.5)
fig_zoom.add_hline(y=0, line_dash="dash", line_color="black")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Minutes1UTC=%{x}<br>Activation_MW=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': 'darkorange', 'dash': 'solid', 'width': 1.5},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '',
              'showlegend': False,
              'type': 'scattergl',
              'x': array(['2026-01-26T21:55:00.000000000', '2026-01-26T21:56:00.000000000',
                          '2026-01-26T21:57:00.000000000', ..., '2026-01-28T21:53:00.000000000',
                          '2026-01-28T21:54:00.000000000', '2026-01-28T21:55:00.000000000'],
                         shape=(2880,), dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('Ieo+AClUYsAMPPcerr9iwMuCiT8KJ2' ... 'XB/9bjZMDfFcH/1uNkwKuYSj/h+mbA'),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'shapes': [{'line': {'color': 'black', 'dash': 'dash'},
                           'type': 'line',
                           'x0': 0,
                           'x1': 1,
                           'xref': 'x domain',
                           'y0': 0,
                           'y1': 0,
                           'yref': 'y'}],
               'template': '...',
               'title': {'text': 'Zoom: Last 48 Hours Microstructure'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Minutes1UTC'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Activation_MW'}}}
})

In [13]:
# Prepare Data
df_eda = df_clean.to_frame()
df_eda['Hour'] = df_eda.index.hour

# Boxplot by Hour
fig_season = px.box(df_eda, x="Hour", y="Activation_MW", 
                    title="Intraday Seasonality: Activation Range by Hour",
                    color="Hour", color_discrete_sequence=px.colors.sequential.Viridis)

fig_season.update_layout(showlegend=False)
fig_season.add_hline(y=0, line_dash="dash", line_color="black")
fig_season.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed